## This code illustrates how to get S&P500 index constituents and their identifiers from CRSP and Compustat
- Researchers used to be able to extract index membership information from Compustat's "comp.idxcst_his" data. Now that S&P pulled this piece of data off from WRDS platform, we have to turn to CRSP to get S&P500 Index membership data. 
- Unfortunately, there is no easy way to uncover constituents info for the other indices covered by "comp.idxcst_his".

In [1]:
##########################################
# S&P 500 Index Constituents             #
# Qingyi (Freda) Song Drechsler          #
# Date: October 2020                     #
##########################################

import pandas as pd
import wrds

### Step 1: Connect to WRDS

In [2]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


### Step 2: Get SP500 Index Membership from CRSP
- I opt for the monthly frequency of the data, but one can choose to work with crsp.dsp500list if more precise date range is needed.

In [3]:
sp500 = conn.raw_sql("""
                        select a.*, b.date, b.ret
                        from crsp.msp500list as a,
                        crsp.msf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/2000'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])

In [4]:
sp500.head()

,permno,start,ending,date,ret
0,46850.0,1976-07-01,2000-12-13,2000-01-31,-0.042329
1,62092.0,1997-01-02,2022-12-30,2000-01-31,0.154167
2,61621.0,1998-10-01,2022-12-30,2000-01-31,0.103813
3,61663.0,1999-09-20,2001-09-20,2000-01-31,-0.055172
4,10107.0,1994-06-07,2022-12-30,2000-01-31,-0.161670


In [5]:
len(sp500)

138451

In [6]:
sp500.tail()

,permno,start,ending,date,ret
138446,65402.0,2017-07-26,2022-12-30,2022-12-30,-0.057623
138447,21207.0,1969-05-15,2022-12-30,2022-12-30,0.005898
138448,65947.0,2009-01-30,2022-12-30,2022-12-30,-0.077150
138449,83443.0,2010-02-16,2022-12-30,2022-12-30,-0.030446
138450,90199.0,2006-11-10,2022-12-30,2022-12-30,-0.033166


### Step 3: Add Other Company Identifiers from CRSP.MSENAMES
- You don't need this step if only PERMNO is required
- This step aims to add TICKER, SHRCD, EXCHCD and etc. 

In [7]:
# Add Other Descriptive Variables

mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt, 
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

In [8]:
mse.sample(5)

,comnam,ncusip,namedt,nameendt,permno,shrcd,exchcd,hsiccd,ticker
48405,MONTEJAS ENERGY RES,61238410,1982-11-01,1984-02-29,55080.0,10.0,3.0,1382.0,MNTJ
88886,SOUTH UMPQUA BANK,84058410,1998-06-16,1999-03-14,86004.0,11.0,3.0,6020.0,UMPQ
93835,WORLDQUEST NETWORKS INC,98156W10,2003-11-14,2004-06-09,87647.0,11.0,3.0,4813.0,WQNI
11803,G T T COMMUNICATIONS INC,36239310,2017-06-01,2019-09-11,13993.0,11.0,1.0,8743.0,GTT
28198,INNOVATOR E T F S TRUST,45783Y84,2021-10-01,2021-11-25,22126.0,73.0,5.0,6726.0,XBOC


In [9]:
len(mse)

111623

In [10]:
# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]
sp500_full.sample(5)

,permno,start,ending,date,ret,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,ticker
376776,89195.0,2002-07-22,2022-12-30,2008-04-30,-0.036971,PRINCIPAL FINANCIAL GROUP INC,74251V10,2006-06-30,2011-06-12,11.0,1.0,6321.0,PFG
807262,14277.0,1965-03-04,2022-12-30,2017-12-29,0.080191,SCHLUMBERGER LTD,80685710,2004-06-10,2020-04-05,12.0,1.0,5651.0,SLB
710020,39642.0,1972-09-14,2022-12-30,2015-09-30,-0.055028,BECTON DICKINSON & CO,07588710,2004-06-10,2022-12-30,11.0,1.0,3841.0,BDX
872126,19393.0,1957-03-01,2022-12-30,2019-06-28,-0.000441,BRISTOL MYERS SQUIBB CO,11012210,2004-06-10,2022-12-30,11.0,1.0,2834.0,BMY
342187,86799.0,2006-06-28,2016-03-03,2007-07-31,-0.096725,CONSOL ENERGY INC,20854P10,2004-06-10,2011-06-09,11.0,1.0,1222.0,CNX


In [11]:
len(sp500_full)

138443

### Step 4: Add Compustat Identifiers
- Link with Compustat's GVKEY and IID if need to work with fundamental data
- Linkage is done through crsp.ccmxpf_linktable

In [12]:
# Linking with Compustat through CCM

ccm=conn.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

In [13]:
# Merge the CCM data with S&P500 data
# First just link by matching PERMNO
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

# Then set link date bounds
sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                        &(sp500ccm['date']<=sp500ccm['linkenddt'])]
sp500ccm.sample(5)

,permno,start,ending,date,ret,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,ticker,gvkey,iid,linktype,linkprim,linkdt,linkenddt
132175,34746.0,1996-03-08,2022-12-30,2017-03-31,-0.069242,FIFTH THIRD BANCORP,31677310,2004-06-10,2022-12-30,11.0,3.0,6711.0,FITB,004640,01,LU,P,1980-01-31,2023-09-27 19:54:52.273055
141364,21573.0,1925-12-31,2022-12-30,2018-05-31,0.046839,INTERNATIONAL PAPER CO,46014610,2004-06-10,2020-02-19,11.0,1.0,2621.0,IP,006104,01,LC,P,1962-01-31,2023-09-27 19:54:52.273055
157901,89217.0,2015-07-09,2022-12-30,2020-08-31,0.041095,ADVANCE AUTO PARTS INC,00751Y10,2010-05-20,2022-12-30,11.0,1.0,5531.0,AAP,145977,01,LC,P,2001-11-29,2023-09-27 19:54:52.273055
165934,76744.0,2013-09-23,2022-12-30,2021-09-30,-0.094363,VERTEX PHARMACEUTICALS INC,92532F10,2004-06-10,2022-12-30,11.0,3.0,2830.0,VRTX,024344,01,LU,P,1991-07-24,2023-09-27 19:54:52.273055
45308,22293.0,1957-03-01,2022-12-30,2005-08-31,0.047769,CORNING INC,21935010,2004-06-10,2020-02-12,11.0,1.0,3229.0,GLW,003532,01,LU,P,1962-01-31,2023-09-27 19:54:52.273055


In [14]:
# Rearrange columns for final output

sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', \
                                  'linktype', 'linkprim', 'linkdt', 'linkenddt'])
sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip', 'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                     'gvkey', 'iid', 'start', 'ending', 'ret']]
sp500ccm.sample(5)

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret
145760,2019-01-31,40539.0,T J X COMPANIES INC NEW,87254010,11.0,1.0,5651.0,TJX,011672,01,1985-09-26,2022-12-30,0.111533
78414,2009-12-31,75100.0,TIFFANY & CO NEW,88654710,11.0,1.0,5944.0,TIF,013646,01,2000-06-21,2021-01-06,0.011481
101624,2013-01-31,40539.0,T J X COMPANIES INC NEW,87254010,11.0,1.0,5651.0,TJX,011672,01,1985-09-26,2022-12-30,0.064311
69910,2008-11-28,12062.0,LABORATORY CORP AMERICA HLDGS,50540R40,11.0,1.0,8071.0,LH,014960,01,2004-11-01,2022-12-30,0.030411
31134,2003-11-28,75510.0,ADOBE SYSTEMS INC,00724F10,11.0,3.0,7372.0,ADBE,012540,01,1997-05-06,2022-12-30,-0.055987


In [15]:
cnt = sp500ccm.groupby(['date'])['permno'].count().reset_index().rename(columns={'permno':'npermno'})
cnt.sample(4)

,date,npermno
4,2000-05-31,500
126,2010-07-30,501
24,2002-01-31,500
236,2019-09-30,501


In [16]:
sp500ccm.sample(10)

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret
154262,2020-02-28,15069.0,MARATHON OIL CORP,56584910,11.0,1.0,1311.0,MRO,007017,01,1925-12-31,2022-12-30,-0.267370
124401,2016-02-29,23026.0,FIRSTENERGY CORP,33793210,11.0,1.0,4911.0,FE,008099,01,1957-07-10,2022-12-30,0.023291
119313,2015-06-30,52090.0,MCCORMICK & CO INC,57978020,11.0,1.0,2099.0,MKC,007146,01,2003-03-21,2022-12-30,0.031210
50724,2006-05-31,65330.0,LEGG MASON INC,52490110,11.0,1.0,6282.0,LM,006653,01,2006-04-24,2016-12-01,-0.190327
76997,2009-10-30,76082.0,CABOT OIL & GAS CORP,12709710,11.0,1.0,1311.0,COG,020548,01,2008-06-23,2022-12-30,0.076084
27413,2003-05-30,58094.0,NORTH FORK BANCORPORATION NY INC,65942410,11.0,1.0,6022.0,NFB,015202,01,2002-07-17,2006-11-30,0.021941
76583,2009-09-30,77730.0,TYSON FOODS INC,90249410,11.0,1.0,2015.0,TSN,010793,01,2005-08-11,2022-12-30,0.053378
143262,2018-08-31,25081.0,COMERICA INC,20034010,11.0,1.0,6021.0,CMA,003231,01,1995-12-01,2022-12-30,0.005570
76169,2009-09-30,86136.0,SEMPRA ENERGY,81685110,11.0,1.0,4932.0,SRE,008272,01,1998-06-30,2022-12-30,0.000598
108334,2013-12-31,75100.0,TIFFANY & CO NEW,88654710,11.0,1.0,5944.0,TIF,013646,01,2000-06-21,2021-01-06,0.044649


## Slice for latest sp500 companies based on dates

Since our analysis time frame is set to be from 2018 to 2022, we tried to filter the sp500ccm 'date' column, we filtered for '2018-01-31', '2022-12-30' to check and found that the list of companies returned are not the same. 

In the end, we just decide to filter on the latest date '2022-12-30', with the columns of interest: 
['date','permno','comnam','ncusip','ticker','gvkey'], and store them to a new dataframe called 'sp500_filtered'

In [17]:
df_start = sp500ccm[sp500ccm['date']=='2018-01-31']
df_start

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret
138338,2018-01-31,89533.0,WYNN RESORTS LTD,98313410,11.0,3.0,7990.0,WYNN,149318,01,2008-11-14,2022-12-30,-0.017795
138339,2018-01-31,81481.0,DOLLAR TREE INC,25674610,11.0,3.0,5331.0,DLTR,031587,01,2011-12-19,2022-12-30,0.071662
138340,2018-01-31,79545.0,BORGWARNER INC,09972410,11.0,1.0,3714.0,BWA,028742,01,2011-12-19,2022-12-30,0.101194
138341,2018-01-31,14776.0,SYNCHRONY FINANCIAL,87165B10,11.0,1.0,6021.0,SYF,020686,01,2015-11-18,2022-12-30,0.027713
138342,2018-01-31,66800.0,AMERICAN INTERNATIONAL GROUP INC,02687478,11.0,1.0,6331.0,AIG,001487,01,1980-02-14,2022-12-30,0.072843
...,...,...,...,...,...,...,...,...,...,...,...,...,...
138949,2018-01-31,85058.0,BOSTON PROPERTIES INC,10112110,18.0,1.0,6798.0,BXP,064925,01,2006-04-03,2022-12-30,-0.048604
138950,2018-01-31,85265.0,S L GREEN REALTY CORP,78440X10,18.0,1.0,6798.0,SLG,065290,01,2015-03-23,2021-03-19,-0.004062
138952,2018-01-31,19166.0,F M C CORP,30249130,11.0,1.0,2812.0,FMC,004510,01,2009-08-19,2022-12-30,-0.035179
138953,2018-01-31,89017.0,WILLIS TOWERS WATSON PUB LTD CO,G9662910,12.0,3.0,8742.0,WLTW,144009,01,2016-01-05,2022-12-30,0.064901


In [18]:
df_end = sp500ccm[sp500ccm['date']=='2022-12-30']
df_end

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret
174525,2022-12-30,17307.0,VICI PROPERTIES INC,92565210,18.0,1.0,6798.0,VICI,032580,01,2022-06-08,2022-12-30,-0.041228
174526,2022-12-30,90038.0,ASSURANT INC,04621X10,11.0,1.0,6411.0,AIZ,157057,01,2007-04-10,2022-12-30,-0.024645
174527,2022-12-30,82307.0,DAVITA INC,23918K10,11.0,1.0,8092.0,DVA,061483,01,2008-07-31,2022-12-30,0.012749
174528,2022-12-30,36003.0,ROLLINS INC,77571110,11.0,1.0,7342.0,ROL,009225,01,2018-10-01,2022-12-30,-0.096439
174530,2022-12-30,17144.0,GENERAL MILLS INC,37033410,11.0,1.0,2099.0,GIS,005071,01,1969-02-13,2022-12-30,-0.016999
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175130,2022-12-30,65402.0,SMITH A O CORP,83186520,11.0,1.0,3639.0,AOS,009771,02,2017-07-26,2022-12-30,-0.057623
175132,2022-12-30,21207.0,NEWMONT CORP,65163910,11.0,1.0,1041.0,NEM,007881,01,1969-05-15,2022-12-30,0.005898
175133,2022-12-30,65947.0,WELLTOWER INC,95040Q10,18.0,1.0,6513.0,HCN,005543,01,2009-01-30,2022-12-30,-0.077150
175134,2022-12-30,83443.0,BERKSHIRE HATHAWAY INC DEL,08467070,11.0,1.0,6331.0,BRK,002176,02,2010-02-16,2022-12-30,-0.030446


In [19]:
df_diff = pd.concat([df_start['gvkey'],df_end['gvkey']]).drop_duplicates(keep=False)
df_diff

138352    165052
138361    143974
138366    004990
138379    174729
138381    010466
           ...  
175114    026061
175116    020748
175119    064630
175120    030004
175129    175404
Name: gvkey, Length: 165, dtype: object

In [20]:
# In the end, we decide to filter for the date '2022-12-30' to obtain a list of the latest sp500 data 
# We keep the relevant rows: 'date','permno','comnam','ncusip','ticker','gvkey'

df_end = sp500ccm[sp500ccm['date']=='2022-12-30']
sp500_filtered = df_end[['date','permno','comnam','ncusip','ticker','gvkey']]
sp500_filtered

,date,permno,comnam,ncusip,ticker,gvkey
174525,2022-12-30,17307.0,VICI PROPERTIES INC,92565210,VICI,032580
174526,2022-12-30,90038.0,ASSURANT INC,04621X10,AIZ,157057
174527,2022-12-30,82307.0,DAVITA INC,23918K10,DVA,061483
174528,2022-12-30,36003.0,ROLLINS INC,77571110,ROL,009225
174530,2022-12-30,17144.0,GENERAL MILLS INC,37033410,GIS,005071
...,...,...,...,...,...,...
175130,2022-12-30,65402.0,SMITH A O CORP,83186520,AOS,009771
175132,2022-12-30,21207.0,NEWMONT CORP,65163910,NEM,007881
175133,2022-12-30,65947.0,WELLTOWER INC,95040Q10,HCN,005543
175134,2022-12-30,83443.0,BERKSHIRE HATHAWAY INC DEL,08467070,BRK,002176


## Map ticker to cik 

After obtaining the latest 'sp500_filtered', we need to further find each company's CIK (Central Index Key) number based on its ticker. Here, we have tried two mapping files:
1. The first one is a mapping file ('dict1)' called 'sec.gov_include_ticker.txt', obatined from SEC's website based on this link:
    * https://www.sec.gov/include/ticker.txt

    The official link for other data resources can be found here:
    * https://www.sec.gov/os/accessing-edgar-data


2. The second one ('dict2') is obtained through a standard python library that I found online called 'sec_cik_mapper', in which the 'StockMapper' can help get the relevant mappings. Here, we want the mapping 'ticker_to_cik'

    The documentation and references for this library can be found here:
    * https://pypi.org/project/sec-cik-mapper/
    
    * https://github.com/jadchaar/sec-cik-mapper
    

After obtaining both dict1 and dict2, I merged them together to get a more complete and comprehensive ticker_to_cik mapping.

But still, after mapping them to our 'sp500_filtered', there are still few 'NA' values in the 'cik' column, in which the tickers were not found in neither of the mapping dictionaries, so I just manually looked them up on web and updated them. 

In the end, I used the '.isna()' function to guarantee that there are no 'NA' values in the 'cik' column, and I stored the completed dataset into a csv file called 'sp500_ciks.csv' for later use.

In [4]:
'''Mapping dictionary 1: dict1'''
# Define an empty dictionary
dict1 = {}

# path to your text file, read from this txt file into a mapping dict1 
file_path = 'sec.gov_include_ticker.txt'  

# Open the text file for reading
with open(file_path, 'r') as file:
    # Loop through each line in the file
    for line in file:
        # Split each line into key and value based on the ':' delimiter
        key, value = line.strip().split('\t')
        
        # Add the key-value pair to the dictionary
        dict1[key.upper()] = value

In [5]:
'''Mapping dictionary 2: dict2'''
# stockMapper
from sec_cik_mapper import StockMapper
# Initialize a stock mapper instance
mapper = StockMapper()
# obtain another mapping standard dict2 from the StockMapper package that we found online 
dict2 = mapper.ticker_to_cik

In [12]:
# merge dict1 and 2
mapping = dict1.copy()
mapping.update(dict2)
# final mapping dictionary
mapping.items()

dict_items([('AAPL', '0000320193'), ('MSFT', '0000789019'), ('BRK-B', '0001067983'), ('UNH', '0000731766'), ('JNJ', '0000200406'), ('V', '0001403161'), ('TSM', '0001046179'), ('XOM', '0000034088'), ('WMT', '0000104169'), ('SPY', '0000884394'), ('JPM', '0000019617'), ('PG', '0000080424'), ('NVDA', '0001045810'), ('LVMUY', '0000824046'), ('MA', '0001141391'), ('CVX', '0000093410'), ('TSLA', '0001318605'), ('LLY', '0000059478'), ('HD', '0000354950'), ('BAC', '0000070858'), ('PFE', '0000078003'), ('KO', '0000021344'), ('ABBV', '0001551152'), ('BABA', '0001577552'), ('NVO', '0000353278'), ('PEP', '0000077476'), ('COST', '0000909832'), ('MRK', '0000310158'), ('TMO', '0000097745'), ('AVGO', '0001730168'), ('DHR', '0000313616'), ('DIS', '0001744489'), ('TM', '0001094517'), ('MCD', '0000063908'), ('ASML', '0000937966'), ('ORCL', '0001341439'), ('ABT', '0000001800'), ('CSCO', '0000858877'), ('NVS', '0001114448'), ('AZN', '0000901832'), ('TMUS', '0001283699'), ('WFC', '0000072971'), ('ACN', '0001

In [25]:
# add a new column cik in sp500_filtered and find the corresponding cik based on ticker using the 'mapping' dictionary 
sp500_filtered['cik'] = sp500_filtered['ticker'].map(mapping)
sp500_filtered

C:\Users\Dell\AppData\Local\Temp\ipykernel_21040\1670885040.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp500_filtered['cik'] = sp500_filtered['ticker'].map(mapping)


,date,permno,comnam,ncusip,ticker,gvkey,cik
174525,2022-12-30,17307.0,VICI PROPERTIES INC,92565210,VICI,032580,0001705696
174526,2022-12-30,90038.0,ASSURANT INC,04621X10,AIZ,157057,0001267238
174527,2022-12-30,82307.0,DAVITA INC,23918K10,DVA,061483,0000927066
174528,2022-12-30,36003.0,ROLLINS INC,77571110,ROL,009225,0000084839
174530,2022-12-30,17144.0,GENERAL MILLS INC,37033410,GIS,005071,0000040704
...,...,...,...,...,...,...,...
175130,2022-12-30,65402.0,SMITH A O CORP,83186520,AOS,009771,0000091142
175132,2022-12-30,21207.0,NEWMONT CORP,65163910,NEM,007881,0001164727
175133,2022-12-30,65947.0,WELLTOWER INC,95040Q10,HCN,005543,NaN
175134,2022-12-30,83443.0,BERKSHIRE HATHAWAY INC DEL,08467070,BRK,002176,NaN


In [26]:
# Adter doing this, there are still some companies that has 'Nan' values in the cik columns
sp500_filtered[sp500_filtered['cik'].isna()==True]

,date,permno,comnam,ncusip,ticker,gvkey,cik
174728,2022-12-30,29946.0,BROWN FORMAN CORP,11563720,BF,002435,NaN
175133,2022-12-30,65947.0,WELLTOWER INC,95040Q10,HCN,005543,NaN
175134,2022-12-30,83443.0,BERKSHIRE HATHAWAY INC DEL,08467070,BRK,002176,NaN


In [31]:
# So I manually searched for their respctive 'cik' and updated them manually
# BROWN FORMAN CORP: 14693
# WELLTOWER INC: 1535385
# BERKSHIRE HATHAWAY INC DEL: 1067983
sp500_filtered.at[174728, 'cik'] = 14693
sp500_filtered.at[175133, 'cik'] = 1535385
sp500_filtered.at[175134,'cik']=1067983

In [32]:
# Now every ticker has a corresponding cik value! 
sp500_filtered[sp500_filtered['cik'].isna()==True]

,date,permno,comnam,ncusip,ticker,gvkey,cik


In [33]:
# Store it in 'sp500_ciks.csv' to later use it in 'download_txt.ipynb'
sp500_filtered.to_csv('sp500_ciks.csv')